In [1]:
from hypex.dataset import Dataset, InfoRole, TreatmentRole, TargetRole, StratificationRole
from hypex.experiments.aa import AATest

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(),
        "gender": StratificationRole(str)
    }, data="data.csv",
)
# data

In [3]:
aa = AATest(n_iterations=2000)
res = aa.execute(data)

In [4]:
res.resume

  TTest aa test KSTest aa test TTest best split KSTest best split  result  \
0        NOT OK         NOT OK               OK                OK  NOT OK   
1        NOT OK         NOT OK               OK                OK  NOT OK   

       feature group  
0  post_spends     0  
1   pre_spends     0  

In [5]:
res.aa_score

                       pass     score
pre_spends TTest 0    False  0.550274
post_spends TTest 0   False  0.551360
pre_spends KSTest 0   False  0.493654
post_spends KSTest 0  False  0.517096

In [6]:
res.best_split

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry    split  
0     E-commerce  control  
1     E-commerce  control  
2  

In [7]:
res.best_split_statistic

       feature group TTest pass  TTest p-value KSTest pass  KSTest p-value
0   pre_spends     0         OK       0.993447          OK        0.999954
1  post_spends     0         OK       0.891229          OK        0.996035

In [8]:
res.experiments

              splitter_id  pre_spends GroupDifference control mean 0  \
0        AASplitter┴rs 0┴                                   486.8074   
1        AASplitter┴rs 1┴                                   486.8542   
2        AASplitter┴rs 2┴                                   487.1430   
3        AASplitter┴rs 3┴                                   487.5133   
4        AASplitter┴rs 4┴                                   486.9905   
...                   ...                                        ...   
1995  AASplitter┴rs 1995┴                                   487.2524   
1996  AASplitter┴rs 1996┴                                   486.8883   
1997  AASplitter┴rs 1997┴                                   487.0833   
1998  AASplitter┴rs 1998┴                                   487.4588   
1999  AASplitter┴rs 1999┴                                   487.4373   

      pre_spends GroupDifference test mean 0  \
0                                   487.3801   
1                                   487